# DATA CLEANING E DATA ENRICHMENT
Il dataset di partenza, già fornito in precedenza, è disponibile [qui](https://www.kaggle.com/datasets/heemalichaudhari/adidas-sales-dataset).

### IMPORT LIBRERIE
Questa cella importa le librerie necessarie al funzionamento del notebook.
Sono già preinstallate in Colab.

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm

### IMPORT DATASET

In [ ]:
df = pd.read_excel("Adidas US Sales Datasets Edit.xlsx")

### COORDINATE GEOGRAFICHE PER CITTÀ E STATI FEDERATI
Questa cella esegue il download delle coordinate delle singole città e dei singoli stati, discriminando tra gli omonimi di Regno Unito e USA.

In [ ]:
# Dataframe temporaneo
coordinates = pd.DataFrame({
    "City": df["City"],
    "State": df["State"],
    "City_state": df["City"] + ", " + df["State"]
    })

coordinates["State"][coordinates["State"] == "Washington"] = "Washington State"
coordinates = coordinates.drop_duplicates()

# Istanza geolocator
geolocator = Nominatim(user_agent="user_name")
location_restrictions = ", United States of America"

# Popolamento
tqdm.pandas(desc="Processing City Latitude")
coordinates["City_latitude"] = coordinates["City_state"].progress_apply(lambda x: (geolocator.geocode(x + location_restrictions, timeout=10).latitude))

tqdm.pandas(desc="Processing City Longitude")
coordinates["City_longitude"] = coordinates["City_state"].progress_apply(lambda x: (geolocator.geocode(x + location_restrictions, timeout=10).longitude))

tqdm.pandas(desc="Processing State Latitude")
coordinates["State_latitude"] = coordinates["State"].progress_apply(lambda x: (geolocator.geocode(x + location_restrictions, timeout=10).latitude))

tqdm.pandas(desc="Processing State Longitude")
coordinates["State_longitude"] = coordinates["State"].progress_apply(lambda x: (geolocator.geocode(x + location_restrictions, timeout=10).longitude))

# Merge con dataframe principale
coordinates["State"][coordinates["State"] == "Washington State"] = "Washington"
df = pd.merge(df, coordinates, on=['City', 'State'], how='left')
df.drop(columns=["City_state"], inplace=True)

### DATA CLEANING E ULTERIORI OPERAZIONI DI ARRICCHIMENTO
Questa cella esegue le correzioni dei valori sbagliati di *Total Sales* e *Operating Profit*, modifica le modalità della variabile *Sales Method* e aggiunge altre colonne di potenziale interesse.



In [ ]:
# Correzione valori sbagliati
df["Total Sales"] = df["Price per Unit"] * df["Units Sold"]
df["Operating Profit"] = df["Total Sales"] * df["Operating Margin"]

# Aggiunta colonne di potenziale interesse
df["Operating Cost"] = df["Total Sales"] - df["Operating Profit"]
df["Margin per unit"] = df["Price per Unit"] * df["Operating Margin"]
df["Price Tier"] = df["Price per Unit"].apply(lambda x: "Low" if x<40 else "High" if x>80 else "Medium")

# Ricodifica di Sales Method
df["Sales Method"] = df["Sales Method"].apply(lambda x: "Physical" if x=="Outlet" or x=="In-store" else x)

# Drop colonne indesiderate
df.drop(columns=["Retailer ID"], inplace=True)

### EXPORT DATASET
Si esegue questa cella per salvare il dataframe modificato prima in cloud poi in locale.

In [ ]:
df.to_excel("Adidas_US_Sales_Dataset_Cleaned.xlsx", index=False)